## Importing Libraries

In [39]:
import os

from delta import *
from delta.tables import *

from dotenv import load_dotenv

import pyspark
from pyspark.sql.types import *
from pyspark.sql import functions as F

## Loading Environment Variables

In [2]:
load_dotenv()

api_key = os.getenv("API_KEY")

## Create Spark Session

In [3]:
#  Create a spark session with Delta
builder = pyspark.sql.SparkSession.builder.appName("esports_tournaments_gold") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")

# Create spark context
spark = configure_spark_with_delta_pip(builder).getOrCreate()
spark.sparkContext.setLogLevel("ERROR")

24/03/02 06:34:46 WARN Utils: Your hostname, pitta resolves to a loopback address: 127.0.1.1; using 192.168.100.7 instead (on interface enp6s0)
24/03/02 06:34:46 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/home/pitta/anaconda3/envs/esports-pipeline/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/pitta/.ivy2/cache
The jars for the packages stored in: /home/pitta/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-c26f3d9c-9667-4fd7-923a-933f6e6e1e5f;1.0
	confs: [default]
	found io.delta#delta-core_2.12;2.4.0 in central
	found io.delta#delta-storage;2.4.0 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
:: resolution report :: resolve 289ms :: artifacts dl 16ms
	:: modules in use:
	io.delta#delta-core_2.12;2.4.0 from central in [default]
	io.delta#delta-storage;2.4.0 from central in [default]
	org.antlr#antlr4-runtime;4.9.3 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   3   |   0   | 

## Load Data

In [84]:
esports_tournaments_df = spark.read.format('delta').load('../../data/silver/esports_tournaments/')
esports_games_genre_df = spark.read.format('delta').load('../../data/silver/esports_games_genre/')
esports_games_awarding_prize_money_df = spark.read.format('delta').load('../../data/silver/esports_games_awarding_prize_money/')

In [87]:
esports_tournaments_df.printSchema()
esports_tournaments_df.orderBy("TournamentId").show()
esports_tournaments_df.count()

root
 |-- Genre: string (nullable = true)
 |-- GameName: string (nullable = true)
 |-- GameId: integer (nullable = true)



+--------------------+--------------------+------+
|               Genre|            GameName|GameId|
+--------------------+--------------------+------+
|            Strategy|        StarCraft II|   151|
|            Strategy|StarCraft: Brood War|   152|
|First-Person Shooter|Halo: Combat Evolved|   153|
|First-Person Shooter|              Halo 2|   154|
|First-Person Shooter|              Halo 3|   155|
|First-Person Shooter|         Halo: Reach|   156|
|First-Person Shooter|          Quake Live|   157|
|            Strategy|        WarCraft III|   158|
|First-Person Shooter|     Quake III Arena|   159|
|First-Person Shooter|          QuakeWorld|   160|
|First-Person Shooter|            Quake II|   161|
|First-Person Shooter|      Counter-Strike|   162|
|Multiplayer Onlin...|Defense of the An...|   163|
|Multiplayer Onlin...|   League of Legends|   164|
|   Role-Playing Game|   World of WarCraft|   165|
|       Fighting Game|Super Street Figh...|   166|
|       Fighting Game|Street Fi

657

In [95]:
esports_tournaments_with_genre_df = esports_tournaments_df.join(esports_games_genre_df.select("GameId", "GameName", "Genre"), on="GameId", how="inner")
esports_tournaments_with_genre_df.orderBy("TournamentId").show()
esports_tournaments_with_genre_df.count()

+------+------------+--------------------+----------+----------+--------------------+--------+-------------+------------+--------+
|GameId|TournamentId|      TournamentName| StartDate|   EndDate|            Location|Teamplay|TotalUSDPrize|    GameName|   Genre|
+------+------------+--------------------+----------+----------+--------------------+--------+-------------+------------+--------+
|   151|        1000|GSL January 2011 ...|2011-01-02|2011-01-29|  Seoul, South Korea|       0|    118364.41|StarCraft II|Strategy|
|   151|        1001|GSL January 2011 ...|2011-01-02|2011-01-29|  Seoul, South Korea|       0|     11567.43|StarCraft II|Strategy|
|   151|        1002|GSL March 2011 (C...|2011-02-21|2011-03-19|  Seoul, South Korea|       0|    116046.45|StarCraft II|Strategy|
|   151|        1003|GSL March 2011 (C...|2011-02-21|2011-03-19|  Seoul, South Korea|       0|     11471.31|StarCraft II|Strategy|
|   151|        1004|GSL World Champio...|2011-03-28|2011-04-09|  Seoul, South Kore

63865

## Join Data

In [ ]:
esports_tournaments_genre_df = esports_tournaments_df.join(esports_games_genre_df.select("GameId", "GameName", "Genre"), on="GameId", how="inner")
esports_tournaments_genre_df.show()

In [ ]:
esports_games_awarding_prize_money_genre_df = esports_games_awarding_prize_money_df.join(esports_games_genre_df.select("GameId", "Genre"), on="GameId", how="inner")
esports_games_awarding_prize_money_genre_df.show()